In [1]:
from bs4 import BeautifulSoup as soup
from bs4 import BeautifulSoup
import requests 
import pandas as pd
from datetime import date
from time import sleep
from calendar import monthrange
import json
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, parser)
    return json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))

request_params = {'Cookie':'__ddg1_=DJi0YgXW8q062VjdNMQN; PHPSESSID=8233258526fbed7e597d89b3720f5e03; new_pf0=1; new_pf10=1; hidetopprjlenta=0; _tm_lt_sid=1663326271744.318568; _ym_uid=1663326273770135271; _ym_d=1663326273; _ym_visorc=w; _ym_isad=1; mindboxDeviceUUID=eecb57cb-7adf-4770-a5f3-b5db571812ef; directCrm-session={"deviceGuid":"eecb57cb-7adf-4770-a5f3-b5db571812ef"}; _ga=GA1.2.1668091564.1663326274; _gid=GA1.2.1612928119.1663326274; _ga_cid=1668091564.1663326274; uechat_3_first_time=1663326274765; uechat_3_mode=0; _gat=1; uechat_3_pages_count=8; uechat_3_disabled=true'
 ,'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

In [5]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : '/Users/nikolay/Desktop/Диплом/final_versions'} ---Здесь укажи куда нужно скачивать
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.maximize_window()

/var/folders/kw/l_ghrx9j4cq256h30xlz7rx40000gn/T/ipykernel_74740/12833210.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [58]:
import numpy as np
from sklearn.decomposition import PCA

In [86]:
#downloading trends

file1 = open('trends_krasnodar.txt', 'r')
Lines = file1.readlines()
 

for line in Lines:
    search,weigt,topic = line.split(",")
    print(search,weigt,topic)
    driver.get("https://trends.google.ru/trends/explore?date=2014-01-01%202023-01-01&geo=RU-KDA&q=" + search)
    sleep(5)
    try:
        driver.find_element("xpath", "//button[@class='widget-actions-item export']").click()
    except:
        driver.get("https://trends.google.ru/trends/explore?date=2014-01-01%202023-01-01&geo=RU-KDA&q=" + search)
        driver.get("https://trends.google.ru/trends/explore?date=2014-01-01%202023-01-01&geo=RU-KDA&q=" + search)
        sleep(5)

        try:
            driver.find_element("xpath", "//button[@class='widget-actions-item export']").click()
        except:
            print("problem with ", search)


Телевизор  0.73  Телерадиотовары 

USB  0.77  Электротовары и другие бытовые приборы 

Монитор  0.77  Электротовары и другие бытовые приборы

Ноутбук  0.57  Персональные компьютеры

Планшет  0.73  Телерадиотовары

Моноблок  0.73  Телерадиотовары

Телефон стационарный  0.72  Средства связи

Смартфон  0.72  Средства связи

Наушники беспроводные  0.73  Телерадиотовары

умные часы  0.73  Телерадиотовары

Масло  0.78  Масло и жиры

Маргарин  0.78  Масло и жиры

Молоко  0.73  Молоко и молочная продукция

Сливки  0.73  Молоко и молочная продукция

Сметана  0.73  Молоко и молочная продукция

Йогурт  0.73  Молоко и молочная продукция

Творог  0.73  Молоко и молочная продукция

Сгущенка  0.73  Молоко и молочная продукция

Сахар  0.69  Сахар

Зефир  0.82  Кондитерские изделия

Карамель  0.82  Кондитерские изделия

Шоколад  0.82  Кондитерские изделия

Конфеты  0.82  Кондитерские изделия

Жевачка  0.82  Кондитерские изделия

Печенье  0.82  Кондитерские изделия

Пряники  0.82  Кондитерские изделия



In [88]:
first = True
i = 0
res = pd.DataFrame()



file1 = open('trends_krasnodar.txt', 'r')
Lines = file1.readlines()
 

for line in Lines:
    search,weigt_new,topic_new = line.split(",")
    if first:
        first = False
        reading_file = "multiTimeline.csv"
        a = pd.read_csv(reading_file, skiprows=1)
        topic_old = topic_new
        weight_old = weigt_new
        i += 1        
    else:
        if topic_new == topic_old:
            reading_file = "multiTimeline (" + str(i)+  ").csv"
            addition = pd.read_csv(reading_file, skiprows=1)
            
            a = a.merge(addition, on='Месяц', how='left')
            i += 1
        else:
            pca = PCA(n_components=1)
            res[topic_old] = list(map(float, pca.fit_transform(scaler.fit_transform(a.drop(["Месяц"], axis=1).dropna(axis=1)))))
            res[topic_old] = res[topic_old] * float(weight_old)
            reading_file = "multiTimeline (" + str(i)+  ").csv"
            a = pd.DataFrame(pd.read_csv(reading_file, skiprows=1))
            topic_old = topic_new
            weight_old = weigt_new
            i += 1

res = res.diff()/res.shift()
res.to_csv("krsnd_trends_with_corr.csv")